In [2]:
#load packages
import matplotlib.pyplot as plt
import pandas as pd 
#from sklearn.preprocessing import StandardScaler 
import numpy as np
from sklearn.metrics import make_scorer, mean_squared_error, confusion_matrix, classification_report, roc_curve, auc
from sklearn import metrics, svm, neighbors, SVR, tree
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.model_selection  import cross_val_score, train_test_split, cross_validate, cross_val_score,KFold,RandomizedSearchCV,RepeatedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix, r2_score
from sklearn.svm import 
import xgboost as xgb

In [10]:
#set up data
df=pd.read_excel('C:/Users/yidan/Desktop/fall/PA/hw3/HW3.xlsx')

#filtered on the purchase with 1, which represent those who have made a purchase
restricted = df[df['Purchase']==1]

# Data prep: I dropped the Purchase, Sending columns for X and 
# include only Spending column for predicting feature
X=restricted.drop(['Purchase', 'Spending'], axis=1)
y=restricted[['Spending']]
#split the train and test dataset by 70% - 30% proportion
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
#linear regression
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
regr = LinearRegression()
line = regr.fit(X_train, np.ravel(y_train)) 

#Techniques: MSE, R^2. For continuous dependent variables, 
#Accuracy, recall and precision are no longer valid on evaluating model performance. 
#Instead, I look on MSE (as to the information it presents, is equivalent to RMSE in this case), 
#which tells the cumulative variances of the prediction compared to ground truth and 
#is more informative than MAE and MAPE, and R^2, 
#which indicates who well the model can explain the dataset. 
mse = metrics.mean_squared_error(y_test, line.predict(X_test))
print('Linear Regression MSE: %f'%mse)
r2 = r2_score(y_test, line.predict(X_test))
print('Linear Regression R2: %f'%r2)

Linear Regression MSE: 25197.158355
Linear Regression R2: 0.482771


In [16]:
#lasso CV with even lower score
lasso = linear_model.LassoCV(cv=10)
Lline = lasso.fit(X_train, np.ravel(y_train)) 
Lr2 = r2_score(y_test, Lline.predict(X_test)) #ground truth, pred value
print('Linear Regression R2: %f'%Lr2)

0.3752463508227345 0.3752463508227345
Linear Regression R2: 0.375246


In [112]:
#k-NN
k_fold = KFold(n_splits=10)
clf_KNN = neighbors.KNeighborsRegressor()
weight_options = ["uniform", "distance"]
params = {'n_neighbors':range(1,20), 'weights':weight_options}
scoring_fnc = make_scorer(mean_squared_error)
grid = GridSearchCV(clf_KNN, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = grid.fit(X_train, y_train)
clf_KNN = grid.best_estimator_

print('best score: %f'%grid.best_score_)
print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_KNN.get_params()[key]))
print('test score: %f'%clf_KNN.score(X_test, y_test))

best score: 44018.415671
best parameters:
n_neighbors: 1
weights: uniform
test score: -0.198022


In [20]:
#XGBoost
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
xg_reg = xgb.XGBRegressor()

params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
#scoring_fnc = make_scorer(mean_squared_error)
#cv_results = GridSearchCV(xg_reg, param_grid=params,scoring=scoring_fnc,cv=3)
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

xg_reg = xg_reg.fit(X_train,y_train)
#clf_xg = grid.best_estimator_

mse = metrics.mean_squared_error(y_test, xg_reg.predict(X_test))
r2 = metrics.r2_score(y_test, xg_reg.predict(X_test))
#for key in params.keys():
#    print('%s: %s'%(key, clf_xg.get_params()[key]))
print(f'XGBoost MSE:{mse}')
print(f'XGBoost r-squared:{r2}')
print(xg_reg)

[12:19:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:19:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:19:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:19:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost MSE:12578.80486955328
XGBoost r-squared:0.6428001900770023
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=

In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
#one possible reason for the high mean_squared_error and low R-squared rate is that when
#the records of people purchasing nothing is included, the '0' amount in the purchasing 
#item will influence the performance of the model.

In [1]:
## SVR
from sklearn.svm import SVR
clf_SVR = SVR()
C=[0.01, 1, 100]
gamma=[100,1,0.01]
kernel=['linear','rbf']
parameters = {'C': C, 'gamma':gamma, 'kernel':kernel}


In [ ]:
##GridSearch cross validation
scoring_fnc = make_scorer(mean_squared_error)
k_fold = KFold(n_splits=5)
grid = RandomizedSearchCV(clf_SVR, param_distributions=parameters,scoring=scoring_fnc,cv=k_fold)

grid = grid.fit(X_train, np.ravel(y_train))
clf_SVR = grid.best_estimator_


In [ ]:
##result
print('best score: %f'%grid.best_score_)
print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_SVR.get_params()[key]))
print('test score: %f'%clf_SVR.score(X_test, y_test))

In [19]:
#KNN & Reg Tree 
models = {
          'KNN': KNeighborsRegressor(),
          'DT': tree.DecisionTreeRegressor()}

models_parameters = {
                     'KNN': {'n_neighbors': range(1,20), 
                             'p': [1, 2, 3]}, #1：manhattan_distance 2：euclidean_distance
                     'DT': {'max_depth': [2, 3, 4], 
                            'min_samples_split': [2, 3, 4, 5]}}
                
cv_method = RepeatedKFold(n_splits=5, n_repeats=3, random_state=999)

fitted_models = {} # this creates an empty dictionary
for m in models: # this will loop over the dictionary keys
    print(f'\nHyperparameter tuning for {m}:')
    gs = GridSearchCV(estimator=models[m], 
                      param_grid=models_parameters[m], 
                      cv=cv_method,
                      verbose=1, 
                      scoring='neg_mean_squared_error')
    gs.fit(X_train, y_train)
    fitted_models[m] = gs
    print(f'Best {m} model: {gs.best_params_}')

#MSE
from sklearn import metrics
for m in fitted_models:
    t_pred = fitted_models[m].predict(X_test)
    mse = metrics.mean_squared_error(y_test, t_pred)
    print(f'MSE of {m} is: {mse}')

#r^2, MAE
from sklearn import metrics
for m in fitted_models:
    t_pred = fitted_models[m].predict(X_test)
    r2 = metrics.r2_score(y_test, t_pred)
    print(f'MAE and r-squared {m} are: {mae}, {r2}')


Hyperparameter tuning for KNN:
Fitting 15 folds for each of 57 candidates, totalling 855 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 855 out of 855 | elapsed:    7.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best KNN model: {'n_neighbors': 18, 'p': 3}

Hyperparameter tuning for DT:
Fitting 15 folds for each of 12 candidates, totalling 180 fits
Best DT model: {'max_depth': 3, 'min_samples_split': 2}
MSE of KNN is: 28332.03218986574
MSE of DT is: 21277.984762829623
MAE and r-squared KNN are: 95.04128240740742, 0.21802135973269554
MAE and r-squared DT are: 80.95318195758557, 0.41271669180096393


[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:    0.8s finished


In [ ]:
#NNT
from keras.models import Sequential
from keras.layers import Dense

def create_model(activation='relu', nb_hidden=10):
    model = Sequential()
    model.add(Dense(nb_hidden, input_dim=23, activation=activation))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
    return model

from keras.wrappers.scikit_learn import KerasRegressor
activations = ['relu', 'tanh', 'sigmoid', 'linear']
nb_hiddens = np.array([100, 200])

param_grid = dict(activation=activations, nb_hidden=nb_hiddens)
model = KerasRegressor(build_fn=create_model, epochs=20, batch_size=256, verbose=0)
scoring_fnc = make_scorer(mean_squared_error)
clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, scoring=scoring_fnc)
res = clf.fit(X_train, y_train)

#results
print(f'NNT best-score ：{res.best_score_}')
print(f'best hyperparameter{res.best_params_}')
mse = metrics.mean_squared_error(y_test, res.predict(X_test))
r2 = metrics.r2_score(y_test, res.predict(X_test))
print(f'NNT MSE:{mse}')
print(f'NNT r-squared:{r2}')

In [ ]:
pip install xgboost

In [ ]:
#XGBoost
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

mse = metrics.mean_squared_error(y_test, res.predict(X_test))
r2 = metrics.r2_score(y_test, res.predict(X_test))
print(xg_reg)
print(f'NNT MSE:{mse}')
print(f'NNT r-squared:{r2}')

In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)